<a href="https://colab.research.google.com/github/arav16112004/Arav-Projects-Repo/blob/externship_project----LLM-document-QA-bot/Copy_of_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index llama-cpp-python chromadb PyMuPDF

In [ ]:
!pip install llama-index-llms-llama-cpp

In [ ]:
!pip install llama-index-embeddings-huggingface

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, fitz
from chromadb.config import Settings
import chromadb

from llama_index.llms.llama_cpp import LlamaCPP # Corrected import path
from llama_index.core import (
    SimpleDirectoryReader,
    StorageContext,
    Document
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import CompactAndRefine

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import os
model_path = "/content/mistral.gguf"
if not os.path.exists(model_path):
    !wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf -O {model_path}
    print(f"Model downloaded to {model_path}")

    from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core import Document


llm = LlamaCPP(
    model_path="/content/mistral.gguf",
    temperature=0.0,
    max_new_tokens=30,
    context_window=4096,
)


llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /content/mistral.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.

In [ ]:
from llama_index.core import SimpleDirectoryReader # Added import here
from llama_index.core.node_parser import SentenceSplitter

reader = SimpleDirectoryReader("/content/drive/MyDrive/ext", recursive=True)
docs = reader.load_data()
splitter = SentenceSplitter(chunk_size=512, chunk_overlap=128)
chunks = splitter.get_nodes_from_documents(docs)

In [ ]:
!pip install llama-index-vector-stores-chroma

In [ ]:
from chromadb.config import Settings
import chromadb

# make sure these come *before* your loop
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Document, StorageContext, VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore

# …then your setup…
settings   = Settings(anonymized_telemetry=False)
client     = chromadb.Client(settings=settings)
collection = client.get_or_create_collection("rag_demo")

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_docs = []
for node in chunks:
    fn = node.source_node.metadata.get("file_path", "").lower()
    if "bank" in fn:      dt="Bank Statement"
    elif "payslip" in fn:  dt="Pay Slip"
    elif "appraisal" in fn:dt="Appraisal Report"
    else:                  dt="Unknown"
    # now Document is defined
    vector_docs.append(Document(text=node.text, metadata={"doc_type": dt}))

# build your Chroma-backed index
vector_store = ChromaVectorStore(chroma_collection=collection)
storage_ctx  = StorageContext.from_defaults(vector_store=vector_store)
index        = VectorStoreIndex.from_documents(vector_docs, storage_context=storage_ctx, embed_model=embed_model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from llama_index.core.response_synthesizers import CompactAndRefine # Added import here

retriever = index.as_retriever(similarity_top_k=3)
reranker  = CompactAndRefine(llm=llm, verbose=True)
qe        = RetrieverQueryEngine(retriever=retriever, response_synthesizer=reranker)

In [ ]:
tests = {
    "Appraisal": "What is the estimated home value?",
    "Bank":      "How much was the last transaction?",
    "Pay":       "What is the total net salary for this month?"
}

for name, q in tests.items():
    print(f"\n== {name} Query ==")
    resp = qe.query(q)
    print("Answer:", resp.response)


== Appraisal Query ==


llama_perf_context_print:        load time =  358610.74 ms
llama_perf_context_print: prompt eval time =  358609.65 ms /  1627 tokens (  220.41 ms per token,     4.54 tokens per second)
llama_perf_context_print:        eval time =    8307.80 ms /    11 runs   (  755.25 ms per token,     1.32 tokens per second)
llama_perf_context_print:       total time =  366924.11 ms /  1638 tokens
Llama.generate: 20 prefix-match hit, remaining 1768 prompt tokens to eval


Answer: 1,918,507 dollars.

== Bank Query ==


llama_perf_context_print:        load time =  358610.74 ms
llama_perf_context_print: prompt eval time =  369465.44 ms /  1768 tokens (  208.97 ms per token,     4.79 tokens per second)
llama_perf_context_print:        eval time =   21180.87 ms /    29 runs   (  730.37 ms per token,     1.37 tokens per second)
llama_perf_context_print:       total time =  390661.07 ms /  1797 tokens
Llama.generate: 14 prefix-match hit, remaining 1579 prompt tokens to eval


Answer: 1,912,000. The last transaction mentioned in the context information is the one with MLS# 3220

== Pay Query ==


...

Answer: 1,918,507 dollars.

== Bank Query ==


llama_perf_context_print:        load time =  358610.74 ms
llama_perf_context_print: prompt eval time =  364684.67 ms /  1768 tokens (  206.27 ms per token,     4.85 tokens per second)
llama_perf_context_print:        eval time =   21645.02 ms /    29 runs   (  746.38 ms per token,     1.34 tokens per second)
llama_perf_context_print:       total time =  386344.26 ms /  1797 tokens
Llama.generate: 14 prefix-match hit, remaining 1579 prompt tokens to eval


Answer: 1,912,000. The last transaction mentioned in the context information is the one with MLS# 3220

== Pay Query ==


llama_perf_context_print:        load time =  358610.74 ms
llama_perf_context_print: prompt eval time =  334158.80 ms /  1579 tokens (  211.63 ms per token,     4.73 tokens per second)
llama_perf_context_print:        eval time =   20583.51 ms /    29 runs   (  709.78 ms per token,     1.41 tokens per second)
llama_perf_context_print:       total time =  354756.98 ms /  1608 tokens


Answer: 9500.00. The total net salary for this month is 9500.00 as per the payslip
